Autor: Brandon Andrés Jiménez Nieto

# **Prueba práctica:**  
## Clasificación de Pinguinos de la Isla Palmer con un perceptrón multicapa (MLP)

### **Objetivo y alcance del trabajo:**

**Objetivo:**

El objetivo de este Jupyter Notebook es desarrollar y evaluar un modelo de clasificación utilizando un Perceptrón Multicapa (MLP). El modelo se entrenará utilizando un conjunto de datos de entrenamiento y luego se utilizará para hacer predicciones en un conjunto de datos de prueba.

**Alcance:**

Este Jupyter Notebook cubrirá los siguientes aspectos:

1. Importación de las bibliotecas necesarias.
2. Carga y exploración inicial de los datos.
3. Preprocesamiento de los datos, si es necesario.
4. División de los datos en conjuntos de entrenamiento y prueba.
5. Creación de un MLPClassifier.
6. Entrenamiento del MLPClassifier con el conjunto de entrenamiento.
7. Uso del MLPClassifier entrenado para hacer predicciones en el conjunto de prueba.
8. Evaluación del rendimiento del modelo.

El conjunto de datos de Palmer Penguins fue introducido en 2020 por Allison Horst, Alison Hill y Kristen Gorman. Este conjunto de datos es a menudo utilizado para tareas de clasificación en aprendizaje automático.

El conjunto de datos contiene 344 pingüinos observados en tres diferentes islas del archipiélago de Palmer, en la Antártida. Los datos fueron recogidos por el proyecto Palmer Long Term Ecological Research y la Fundación Nacional para la Ciencia de los Estados Unidos.

Cada pingüino se clasifica en una de las tres especies: Adelie, Chinstrap y Gentoo. Para cada pingüino, el conjunto de datos incluye siete características:

1. Especie: Adelie, Chinstrap, Gentoo.
2. Isla: Torgersen, Biscoe, Dream.
3. Longitud del pico (mm).
4. Profundidad del pico (mm).
5. Longitud de la aleta (mm).
6. Masa corporal (g).
7. Sexo: masculino, femenino.

El objetivo principal al trabajar con este conjunto de datos es a menudo construir un modelo de clasificación que pueda predecir la especie de un pingüino basándose en las características que se tienen en la base de datos.

### **Fase I:** Carga y procesamiento de la base de datos

In [1]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Cargamos la base de datos 'palmer_penguins.csv' en un DataFrame de Pandas de nombre 'pinguinos'
pinguinos = pd.read_csv('D:/Archivos de Usuario/Documents/Artificial-Intelligence/Datasets/palmer_penguins.csv')

# Mostremos las primeras 5 filas de la base de datos
pinguinos.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [3]:
# Usamos el método 'isnull()' para identificar los valores nulos y el método 'sum()' para sumarlos
pinguinos.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [4]:
# Usamos el método 'dropna()' para eliminar las filas con valores nulos del DataFrame 'pinguinos'
pinguinos = pinguinos.dropna()

# Usamos el método 'head()' para mostrar las primeras 5 filas de la base de datos
pinguinos.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE


In [5]:
# Usamos el método 'unique()' para encontrar los valores únicos en la columna 'sex'
pinguinos['sex'].unique()

array(['MALE', 'FEMALE', '.'], dtype=object)

In [6]:
# Eliminamos las filas con el valor '.' en la columna 'sex' usando el método de filtrado por comparación negada '!='
pinguinos = pinguinos[pinguinos['sex'] != '.']

### **Fase II:** Preparamos los datos para ser usados en un perceptrón multicapa (MLP)

In [49]:
# Aislamos las variables predictoras que son todas menos la variable objetivo
# Usamos el método 'drop()' para eliminar la columna 'species' del DataFrame 'pinguinos'
X = pinguinos.drop('species', axis=1)

# Convertimos las variables categóricas en variables numéricas mediante el método 'LabelEncoder()'
# Usamos el método 'apply()' para aplicar la función 'LabelEncoder().fit_transform' a cada columna de 'X'
X = X.apply(LabelEncoder().fit_transform)

# Mostremos las primeras 5 filas de la base de datos
X.head()

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,2,41,56,5,30,1
1,2,44,43,10,32,0
2,2,50,49,19,11,0
4,2,21,62,17,18,0
5,2,43,73,14,26,1


In [50]:
# Aislamos la variable objetivo 'species' en la variable 'y'
y = pinguinos['species']

# Mostramos las primeras 5 filas de la base de datos
y.head()

0    Adelie
1    Adelie
2    Adelie
4    Adelie
5    Adelie
Name: species, dtype: object

In [9]:
# Dividimos los datos 'X' y 'y' en entrenamiento y prueba con el método 'train_test_split'
# Usamos el 0.2 (20%) de los datos para prueba
# Guardamos los datos de entrenamiento en las variables 'X_train' y 'y_train'
# Guardamos los datos de prueba en las variables 'X_test' y 'y_test'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### **Fase III:** Clasificación de pinguinos con un perceptrón multicapa (MLP)

In [11]:
# Creamos un clasificador de tipo MLPClassifier con 2 capas ocultas de 100 neuronas cada una
# Usamos la función de activación 'relu' y el optimizador 'adam'
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), activation='relu', solver='adam')

# Usamos el método 'fit()' para entrenar el clasificador
# Entrenamos el clasificador con los datos de entrenamiento 'X_train' y 'y_train'
mlp.fit(X_train, y_train)

# Predecimos las etiquetas de los datos de prueba 'X_test' usando el método 'predict()' del clasificador entrenado
y_pred = mlp.predict(X_test)
y_pred

array(['Adelie', 'Adelie', 'Chinstrap', 'Gentoo', 'Chinstrap', 'Adelie',
       'Chinstrap', 'Gentoo', 'Chinstrap', 'Gentoo', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Gentoo', 'Gentoo', 'Gentoo', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Chinstrap', 'Adelie', 'Gentoo',
       'Adelie', 'Gentoo', 'Adelie', 'Adelie', 'Chinstrap', 'Gentoo',
       'Chinstrap', 'Chinstrap', 'Adelie', 'Gentoo', 'Adelie',
       'Chinstrap', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Chinstrap',
       'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Chinstrap',
       'Adelie', 'Gentoo', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie',
       'Gentoo', 'Chinstrap', 'Gentoo', 'Gentoo', 'Chinstrap', 'Gentoo',
       'Chinstrap', 'Adelie', 'Chinstrap', 'Adelie', 'Adelie',
       'Chinstrap', 'Gentoo', 'Gentoo'], dtype='<U9')

In [12]:
# Calculamos la precisión del modelo de clasificación 'mlp' con el método 'accuracy_score'
# Usamos los datos 'y_test' y 'y_pred' como argumentos de la función 'accuracy_score'
# Guardamos el resultado en la variable 'accuracy'
accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0

In [13]:
# Mostramos la matriz de confusión del modelo de clasificación 'mlp' con el método 'confusion_matrix'
# Usamos los datos 'y_test' y 'y_pred' como argumentos de la función 'confusion_matrix'
confusion_matrix(y_test, y_pred)

array([[25,  0,  0],
       [ 0, 16,  0],
       [ 0,  0, 26]], dtype=int64)

In [14]:
# Imprimimos el reporte de clasificación del modelo de clasificación 'mlp' con el método 'classification_report'
# Usamos los datos 'y_test' y 'y_pred' como argumentos de la función 'classification_report'
# Imprimimos el reporte de clasificación en la consola con la función 'print'
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Adelie       1.00      1.00      1.00        25
   Chinstrap       1.00      1.00      1.00        16
      Gentoo       1.00      1.00      1.00        26

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67



In [15]:
# Hacemos una prueba de predicción de especie con valores de un nuevo pinguino (no visto por el modelo)
# Creamos una lista con los valores de las variables predictoras del nuevo pinguino
# Puedes usar estos valores de ejemplo o cambiarlos por los valores aleatorios de un nuevo pinguino:
#   island = 1 (Biscoe)
#   culmen_length_mm = 45
#   culmen_depth_mm = 58
#   flipper_length_mm = 11
#   body_mass_g = 22
#   sex = 0 (FEMALE)
# Guardamos la lista en la variable 'nuevo_pinguino'
nuevo_pinguino = [[1, 45, 58, 11, 22, 0]]

# Hacemos una predicción de la especie del 'nuevo_pinguino' con el método 'predict' del modelo 'mlp'
# Guardamos la predicción en la variable 'nueva_prediccion'
nueva_prediccion = mlp.predict(nuevo_pinguino)
nueva_prediccion

d:\Archivos de Usuario\Documents\Artificial-Intelligence\ai_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array(['Adelie'], dtype='<U9')

In [16]:
# Guardar los pesos (coeficientes) de las capas del modelo MLP en archivos CSV
for i, coef in enumerate(mlp.coefs_):
    np.savetxt(f"pesos_capa_{i+1}.csv", coef, delimiter=",")

# Guardar los sesgos (bias) de las capas del modelo MLP en archivos CSV
for i, intercept in enumerate(mlp.intercepts_):
    np.savetxt(f"sesgos_capa_{i+1}.csv", intercept, delimiter=",")

# Esto generará archivos pesos_capa_1.csv, pesos_capa_2.csv, etc., que contienen los pesos de cada capa del modelo.
# Y archivos sesgos_capa_1.csv, sesgos_capa_2.csv, etc., que contienen los sesgos de cada capa del modelo.


# Usar estos archivos en Excel para replicar la inferencia del modelo con el nuevo pinguino.

# Nota 1: Asegúrate de que el modelo 'mlp' ya ha sido entrenado antes de ejecutar este código.
# Nota 2: MLPClassifier usa ReLU en las capas ocultas y sigmoid en la salida (para clases binarias) ó softmax en la salida (si tienes más de 2 clases).
# Nota 3: El orden de los logits de salida corresponde al orden alfabético de las clases en y. 

### **Fase IV:** Reproducir la inferencia con los CSV

In [18]:
# Utilidades
def relu(x):
    return np.maximum(0, x)

def softmax(z):
    # z: vector (K,)
    z = np.asarray(z, dtype=float)
    z = z - np.max(z)  # estabilización numérica
    e = np.exp(z)
    return e / e.sum()

In [20]:
# Cargar los pesos/sesgos exportados
W1 = np.loadtxt("pesos_capa_1.csv", delimiter=",")   # forma: (n_features, n_hidden1)
b1 = np.loadtxt("sesgos_capa_1.csv", delimiter=",")  # forma: (n_hidden1,)
W2 = np.loadtxt("pesos_capa_2.csv", delimiter=",")   # forma: (n_hidden1, n_hidden2)
b2 = np.loadtxt("sesgos_capa_2.csv", delimiter=",")  # forma: (n_hidden2,)

# Capa de salida
W3 = np.loadtxt("pesos_capa_3.csv", delimiter=",")   # forma: (n_hidden2, n_classes)
b3 = np.loadtxt("sesgos_capa_3.csv", delimiter=",")  # forma: (n_classes,)

# Preparación del vector
x = np.array([1, 45, 58, 11, 22, 0], dtype=float)  # (island, culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, sex)

In [21]:
# Forward pass manual con ReLU en ocultas y Softmax en salida
z1 = x @ W1 + b1              # (n_hidden1,)
h1 = relu(z1)

z2 = h1 @ W2 + b2             # (n_hidden2,)
h2 = relu(z2)

z3 = h2 @ W3 + b3             # (n_classes,)
probs_np = softmax(z3)        # distribución de probas (Softmax)


In [22]:

# Comparar con scikit-learn para validar
probs_sklearn = mlp.predict_proba([x])[0]

print("Orden de clases (alfabético) -> mlp.classes_:", mlp.classes_)
print("Probas (NumPy/CSV):          ", np.round(probs_np, 6))
print("Probas (scikit-learn):        ", np.round(probs_sklearn, 6))
print("Coinciden (≈): ", np.allclose(probs_np, probs_sklearn, atol=1e-5))

Orden de clases (alfabético) -> mlp.classes_: ['Adelie' 'Chinstrap' 'Gentoo']
Probas (NumPy/CSV):           [1. 0. 0.]
Probas (scikit-learn):         [1. 0. 0.]
Coinciden (≈):  True


d:\Archivos de Usuario\Documents\Artificial-Intelligence\ai_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [24]:
print("\nFormas de matrices/vectores:")
print("W1:", W1.shape, " b1:", b1.shape)
print("W2:", W2.shape, " b2:", b2.shape)
print("W3:", W3.shape, " b3:", b3.shape)
print("x :", x.shape,  " -> logits salida (z3):", z3.shape)


Formas de matrices/vectores:
W1: (6, 100)  b1: (100,)
W2: (100, 100)  b2: (100,)
W3: (100, 3)  b3: (3,)
x : (6,)  -> logits salida (z3): (3,)


### **Fase V:** Generar Excel para replicar la inferencia del MLP (ReLU + Softmax)

In [27]:
# pip install xlsxwriter

In [ ]:
import os
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
from sklearn.preprocessing import LabelEncoder

# pinguinos : DataFrame limpio
# mlp       : MLPClassifier entrenado con todas las columnas label-encoded
FEATURES = ["island", "culmen_length_mm", "culmen_depth_mm",
            "flipper_length_mm", "body_mass_g", "sex"]

# 1) Pesos/sesgos del modelo
coefs = mlp.coefs_
bias  = mlp.intercepts_
W1, W2, W3 = coefs
b1, b2, b3 = bias

n_features = W1.shape[0]
n_hidden1  = W1.shape[1]
n_hidden2  = W2.shape[1]
n_classes  = W3.shape[1]
class_order = mlp.classes_.tolist()

# 2) RE-construir LabelEncoder por CADA columna
# (igual que con .apply(LabelEncoder().fit_transform))
encoders = {}
maps = {}       # col -> list[(valor, codigo)]
for col in FEATURES:
    le = LabelEncoder().fit(pinguinos[col])
    encoders[col] = le
    vals = le.classes_.tolist()
    codes = le.transform(vals).tolist()
    # emparejar preservando tipo: texto/num
    pair_list = list(zip(vals, codes))
    maps[col] = pair_list

# Valores iniciales SEGUROS (deben existir en clases_ de cada encoder)
# Usamos la moda de cada columna (si hay múltiples, tomamos la primera)
init_values = []
for col in FEATURES:
    mode_val = pinguinos[col].mode(dropna=True).iloc[0]
    # Garantizar tipo numérico donde aplique
    if isinstance(mode_val, (int, float, np.integer, np.floating)):
        init_values.append(float(mode_val))
    else:
        init_values.append(str(mode_val))

# 3) Crear Excel
excel_path = "mlp_penguins_inferencia.xlsx"
if os.path.exists(excel_path):
    os.remove(excel_path)

wb = xlsxwriter.Workbook(excel_path)

# Estilos
fmt_title  = wb.add_format({"bold": True, "font_size": 12})
fmt_header = wb.add_format({"bold": True, "bg_color": "#EFEFEF", "border": 1})
fmt_border = wb.add_format({"border": 1})
fmt_num    = wb.add_format({"num_format": "0.000000"})
fmt_pct    = wb.add_format({"num_format": "0.0%"})
fmt_ok     = wb.add_format({"bg_color": "#E6F3E6"})
fmt_note   = wb.add_format({"italic": True, "font_color": "#666666"})

# Helpers
def cell_abs(sheet, r, c):
    return f"{sheet}!{xl_rowcol_to_cell(r, c, row_abs=True, col_abs=True)}"

def range_abs(sheet, r1, c1, r2, c2):
    return f"{sheet}!{xl_rowcol_to_cell(r1,c1,True,True)}:{xl_rowcol_to_cell(r2,c2,True,True)}"

# 4) Hoja Meta: clases y mapas por columna
meta = wb.add_worksheet("Meta")
r = 0
meta.write(r, 0, "Autor:", fmt_header); meta.write(r, 1, "Brandon Andrés Jiménez Nieto"); r += 2
meta.write(r, 0, "Clases (orden mlp.classes_):", fmt_header); r += 1
classes_row = r
meta.write_row(classes_row, 0, class_order, fmt_border)
r = classes_row + 2

# Guardar posiciones (valor, código) por columna para usar en MATCH/INDEX
map_pos = {}  # col -> (r_ini, c_val=0, r_fin, c_code=1)
for col in FEATURES:
    meta.write(r, 0, f"Mapa '{col}' (valor→code)", fmt_header); r += 1
    meta.write_row(r, 0, [f"{col}_valor", f"{col}_code"], fmt_header); r += 1
    r_ini = r
    for val, code in maps[col]:
        # Escribimos valor con el tipo correcto
        if isinstance(val, (int, float, np.integer, np.floating)):
            meta.write_number(r, 0, float(val), fmt_border)
        else:
            meta.write(r, 0, str(val), fmt_border)
        meta.write_number(r, 1, int(code), fmt_border)
        r += 1
    r_fin = r - 1
    map_pos[col] = (r_ini, 0, r_fin, 1)
    r += 1  # línea en blanco entre mapas

# 5) Hoja Pesos
wsW = wb.add_worksheet("Pesos")
# W1
wsW.write(0, 0, "W1 (n_features x n_hidden1)", fmt_title)
for i in range(W1.shape[0]):
    wsW.write_row(1 + i, 0, W1[i, :].tolist(), fmt_num)
b1_title = 1 + W1.shape[0] + 1
wsW.write(b1_title, 0, "b1 (n_hidden1)", fmt_title)
b1_row = b1_title + 1
wsW.write_row(b1_row, 0, b1.tolist(), fmt_num)

# W2
W2_title = b1_row + 2
wsW.write(W2_title, 0, "W2 (n_hidden1 x n_hidden2)", fmt_title)
for i in range(W2.shape[0]):
    wsW.write_row(W2_title + 1 + i, 0, W2[i, :].tolist(), fmt_num)
b2_title = W2_title + 1 + W2.shape[0] + 1
wsW.write(b2_title, 0, "b2 (n_hidden2)", fmt_title)
b2_row = b2_title + 1
wsW.write_row(b2_row, 0, b2.tolist(), fmt_num)

# W3
W3_title = b2_row + 2
wsW.write(W3_title, 0, "W3 (n_hidden2 x n_classes)", fmt_title)
for i in range(W3.shape[0]):
    wsW.write_row(W3_title + 1 + i, 0, W3[i, :].tolist(), fmt_num)
b3_title = W3_title + 1 + W3.shape[0] + 1
wsW.write(b3_title, 0, "b3 (n_classes)", fmt_title)
b3_row = b3_title + 1
wsW.write_row(b3_row, 0, b3.tolist(), fmt_num)

# Rangos absolutos de Pesos
W1_r1, W1_c1 = 1, 0
W1_r2, W1_c2 = W1_r1 + W1.shape[0] - 1, W1_c1 + W1.shape[1] - 1
W2_r1, W2_c1 = W2_title + 1, 0
W2_r2, W2_c2 = W2_r1 + W2.shape[0] - 1, W2_c1 + W2.shape[1] - 1
W3_r1, W3_c1 = W3_title + 1, 0
W3_r2, W3_c2 = W3_r1 + W3.shape[0] - 1, W3_c1 + W3.shape[1] - 1

# 6) Hoja Inputs + Forward
ws = wb.add_worksheet("Inputs")
ws.write(0, 0, "Inferencia MLP (ReLU + Softmax) — Palmer Penguins", fmt_title)
ws.write(1, 0, "Cambia los inputs; la predicción se actualiza al instante.", fmt_note)

# 6.1 Inputs crudos (EDITABLES)
ws.write(3, 0, "Inputs crudos", fmt_header)
ws.write_row(4, 0, FEATURES, fmt_header)
ws.write_row(5, 0, init_values, fmt_border)

# Validación de lista para TODAS las columnas (por el LabelEncoded)
for col in FEATURES:
    r1, c1, r2, c2 = map_pos[col]
    # los valores están en la columna A (0) del mapa
    lst = f"=Meta!$A${r1+1}:$A${r2+1}"
    j = FEATURES.index(col)
    ws.data_validation(5, j, 5, j, {'validate': 'list', 'source': lst})

# 6.2 Inputs codificados (lo que consumió el MLP al entrenar: códigos 0..k)
ws.write(7, 0, "Inputs codificados (TODAS las columnas fueron LabelEncoded)", fmt_header)
ws.write_row(8, 0, FEATURES, fmt_header)
enc_row = 9

def enc_formula_for(col, raw_cell):
    r1, c1, r2, c2 = map_pos[col]
    rng_val  = f"Meta!$A${r1+1}:$A${r2+1}"
    rng_code = f"Meta!$B${r1+1}:$B${r2+1}"
    # Código = INDEX(code_map, MATCH(valor_crudo, val_map, 0))
    return f"=INDEX({rng_code}, MATCH({raw_cell}, {rng_val}, 0))"

for j, col in enumerate(FEATURES):
    raw_cell = xl_rowcol_to_cell(5, j)
    ws.write_formula(enc_row, j, enc_formula_for(col, raw_cell), fmt_border)

# 6.3 Forward pass (usa directamente la fila 'enc_row' como X)
x_row_range = f"{xl_rowcol_to_cell(enc_row, 0)}:{xl_rowcol_to_cell(enc_row, n_features-1)}"

# Capa 1
ws.write(11, 0, "Capa 1 — z1; h1=ReLU(z1)", fmt_header)
z1_hdr = 12; z1_row = 13
ws.write_row(z1_hdr, 0, [f"z1_{i+1}" for i in range(n_hidden1)], fmt_header)
for j in range(n_hidden1):
    W1_col = range_abs("Pesos", W1_r1, W1_c1 + j, W1_r2, W1_c1 + j)
    b1_j   = cell_abs("Pesos", b1_row, j)
    ws.write_formula(z1_row, j, f"=MMULT({x_row_range}, {W1_col}) + {b1_j}", fmt_num)

h1_hdr = z1_row + 2; h1_row = h1_hdr + 1
ws.write_row(h1_hdr, 0, [f"h1_{i+1}" for i in range(n_hidden1)], fmt_header)
for j in range(n_hidden1):
    ws.write_formula(h1_row, j, f"=MAX(0, {xl_rowcol_to_cell(z1_row, j)})", fmt_num)

# Capa 2
ws.write(h1_row + 2, 0, "Capa 2 — z2; h2=ReLU(z2)", fmt_header)
z2_hdr = h1_row + 3; z2_row = z2_hdr + 1
ws.write_row(z2_hdr, 0, [f"z2_{i+1}" for i in range(n_hidden2)], fmt_header)

h1_range = f"{xl_rowcol_to_cell(h1_row,0)}:{xl_rowcol_to_cell(h1_row,n_hidden1-1)}"
for j in range(n_hidden2):
    W2_col = range_abs("Pesos", W2_r1, W2_c1 + j, W2_r2, W2_c1 + j)
    b2_j   = cell_abs("Pesos", b2_row, j)
    ws.write_formula(z2_row, j, f"=MMULT({h1_range}, {W2_col}) + {b2_j}", fmt_num)

h2_hdr = z2_row + 2; h2_row = h2_hdr + 1
ws.write_row(h2_hdr, 0, [f"h2_{i+1}" for i in range(n_hidden2)], fmt_header)
for j in range(n_hidden2):
    ws.write_formula(h2_row, j, f"=MAX(0, {xl_rowcol_to_cell(z2_row, j)})", fmt_num)

# Salida + Softmax estable
ws.write(h2_row + 2, 0, "Salida — z3 (logits) y Probabilidades (Softmax)", fmt_header)
z3_hdr = h2_row + 3; z3_row = z3_hdr + 1
ws.write_row(z3_hdr, 0, [f"z3_{k+1}" for k in range(n_classes)], fmt_header)

h2_range = f"{xl_rowcol_to_cell(h2_row,0)}:{xl_rowcol_to_cell(h2_row,n_hidden2-1)}"
for k in range(n_classes):
    W3_col = range_abs("Pesos", W3_r1, W3_c1 + k, W3_r2, W3_c1 + k)
    b3_k   = cell_abs("Pesos", b3_row, k)
    ws.write_formula(z3_row, k, f"=MMULT({h2_range}, {W3_col}) + {b3_k}", fmt_num)

probs_hdr = z3_row + 2
ws.write_row(probs_hdr, 0, [f"p({c})" for c in class_order], fmt_header)
nums_row   = probs_hdr + 1
final_row  = nums_row + 1
aux_max_col = n_classes + 2
aux_den_col = n_classes + 3

z3_vec = f"{xl_rowcol_to_cell(z3_row,0)}:{xl_rowcol_to_cell(z3_row,n_classes-1)}"
ws.write(probs_hdr, aux_max_col, "max_z3", fmt_header)
ws.write(probs_hdr, aux_den_col, "den", fmt_header)

max_cell = xl_rowcol_to_cell(nums_row, aux_max_col)
den_cell = xl_rowcol_to_cell(nums_row, aux_den_col)

ws.write_formula(nums_row, aux_max_col, f"=MAX({z3_vec})", fmt_num)
for k in range(n_classes):
    ws.write_formula(nums_row, k, f"=EXP({xl_rowcol_to_cell(z3_row,k)}-{max_cell})", fmt_num)

nums_range = f"{xl_rowcol_to_cell(nums_row,0)}:{xl_rowcol_to_cell(nums_row,n_classes-1)}"
ws.write_formula(nums_row, aux_den_col, f"=SUM({nums_range})", fmt_num)

for k in range(n_classes):
    ws.write_formula(final_row, k, f"={xl_rowcol_to_cell(nums_row,k)}/{den_cell}", fmt_pct)

# Predicción final
pred_row = final_row + 2
ws.write(pred_row, 0, "Predicción (clase):", fmt_header)
classes_rng = f"Meta!{xl_rowcol_to_cell(classes_row,0,True,True)}:{xl_rowcol_to_cell(classes_row,n_classes-1,True,True)}"
final_probs = f"{xl_rowcol_to_cell(final_row,0)}:{xl_rowcol_to_cell(final_row,n_classes-1)}"
ws.write_formula(pred_row, 1, f"=INDEX({classes_rng}, MATCH(MAX({final_probs}), {final_probs}, 0))", fmt_ok)

wb.close()
print(f"[OK] Excel dinámico creado: {excel_path}")


[OK] Excel dinámico creado: mlp_penguins_inferencia.xlsx


![alt text](image-1.png)
![alt text](image-3.png)
![alt text](image-4.png)
![alt text](image-5.png)
![alt text](image-6.png)

![alt text](image-2.png)

![alt text](image.png)

**Useful Resources:**
- https://support.microsoft.com/es-es/office/función-mmult-40593ed7-a3cd-4b6b-b9a3-e4ad3c7245eb